# [Perceptron](https://en.wikipedia.org/wiki/Perceptron)

## logic gates using perceptron

XOR is expressed as AND( NAND(x1,x2), OR(x1,x2) ) and can be implemented as Multi Layer Perceptron

In [ ]:
# AND
def AND(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    tmp = x1*w1 + x2*w2
    if tmp >= theta:
        return 1
    else:
        return 0
    
def OR(x1, x2):
    w1, w2, theta = 0.1, 0.1, 0.5
    tmp = x1*w1 + x2*w2
    if tmp==0:
        return 0
    else:
        return 1

def NAND(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    tmp = x1*w1 + x2*w2
    if tmp < theta:
        return 1
    else:
        return 0
    
def XOR(x1, x2):
    in1 = OR(x1,x2)
    in2 = NAND(x1,x2)
    return(AND(in1, in2))
    
# test
print("AND")
print(AND(0, 0))
print(AND(0, 1))
print(AND(1, 0))
print(AND(1, 1))

print("OR")
print(OR(0, 0))
print(OR(0, 1))
print(OR(1, 0))
print(OR(1, 1))

print("NAND")
print(NAND(0, 0))
print(NAND(0, 1))
print(NAND(1, 0))
print(NAND(1, 1))

print("XOR")
print(XOR(0, 0))
print(XOR(0, 1))
print(XOR(1, 0))
print(XOR(1, 1))

# NN

* パーセプトロンの例では重みを（真理値表を踏まえて）人手で決めた。ニューラルネットワークを用いれば、適切な重みパラメータをデータから自動で学習できる。
* パーセプトロンでは活性化関数にステップ関数を利用している。活性化関数をステップ関数から別の関数に変更する事でNNへ。
* ステップ関数ではほとんどの場所（0以外の点）において傾きが0であるのに対して、シグモイド関数の傾きは0にならない。この性質がニューラルネットワークの学習にとって好ましい性質となっている。

In [ ]:
# Example of step function and sigmoid function
import numpy as np
import matplotlib.pylab as plt

def step_function(x):
    y = x>0
    y = y.astype(np.int)
    return y

def sigmoid(x):
    return 1/(1+np.exp(-x))

# Step function
x = np.arange(-5.0, 5.0, 0.1)
y = step_function(x)
plt.plot(x,y)
plt.ylim(-0.1, 1.1)
plt.show()

# Sigmoid function
x = np.arange(-5.0, 5.0, 0.1)
y = sigmoid(x)
plt.plot(x,y)
plt.ylim(-0.1, 1.1)
plt.show()

#### Softmax 実装上の Tip

定義式通りにコードを書くとオーバーフローが発生する可能性があるので、入力信号の最大の値を減算する事が一般的に行われる。

In [ ]:
import numpy as np

# naive implementation
a = np.array([1010, 1000, 990])
np.exp(a) / np.sum(np.exp(a))
# result:
# array([nan, nan, nan])

# modified implementation
c = np.max(a)
np.exp(a-c) / np.sum(np.exp(a-c))
# result
# array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

# softmax の出力の総和は 1.0 になる
# つまり、確率として解釈する事が可能となっている
sum(np.exp(a-c) / np.sum(np.exp(a-c)))

## MNIST

※なお、クラス分類では出力層のソフトマックスは省略可能。
これは、exp(x)が単調増加する関数なので、ソフトマックスを使っても使わなくても出力の一番大きなニューロンの場所が変わらないため。
実際に、指数関数の計算はコンピュータリソースが必要となるため出力層のソフトマックスは省略する事が多いようだ。

In [ ]:
import sys, os
import numpy as np
from PIL import Image
import pickle
sys.path.append("./dlscratch")
from dataset.mnist import load_mnist
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(a):
    c = np.max(a)
    return (np.exp(a-c) / np.sum(np.exp(a-c)))

def img_show(img):
    #pil_img = Image.fromarray(np.uint8(img))
    #pil_img.show()
    plt.figure(3)
    plt.clf()
    plt.imshow(img)
    #plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
    return x_train, t_train, x_test, t_test

def init_network():
    with open("./dlscratch/ch03/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        return network
    
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    #print(a3.shape)
    y = softmax(a3)
    
    return y
    
def main():
    x_train, t_train, x_test, t_test = get_data()
    network = init_network()
    
    img = x_train[0].reshape(28, 28)
    img_show(img)
    
    accuracy_cnt = 0
    for i in range(len(x_train)):
        y = predict(network, x_train[i])
        
        if t_train[i] == np.argmax(y):
            accuracy_cnt += 1
            
    print("Accuracy: {}".format(accuracy_cnt / len(x_train)))
    
if __name__ == '__main__':
    main()

In [ ]:
# Batch version
import sys, os
import numpy as np
from PIL import Image
import pickle
sys.path.append("./dlscratch")
from dataset.mnist import load_mnist
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(a):
    c = np.max(a)
    return (np.exp(a-c) / np.sum(np.exp(a-c)))

def img_show(img):
    #pil_img = Image.fromarray(np.uint8(img))
    #pil_img.show()
    plt.figure(3)
    plt.clf()
    plt.imshow(img)
    #plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
    return x_train, t_train, x_test, t_test

def init_network():
    with open("./dlscratch/ch03/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        return network
    
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    #print(a3.shape)
    y = softmax(a3)
    
    return y
    
def main():
    x_train, t_train, x_test, t_test = get_data()
    network = init_network()

    batch_size = 100
    accuracy_cnt = 0
    
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i+batch_size]
        y = predict(network, x_batch)
        
        p = np.argmax(y, axis=1)
        accuracy_cnt += np.sum(p == t_train[i:i+batch_size])
            
    print("Accuracy: {}".format(accuracy_cnt / len(x_train)))
    
if __name__ == '__main__':
    main()

# Learning in NN

「訓練データを使って学習する」＝訓練データに対する損失関数の値を求め、その値をできるだけ小さくするようなパラメータを探し出すこと

### cross entropy loss

$E = - \sum_k{t_k log y_k}$
※$k$はk次元目の値を意味する

ここで、$y_k$はニューラルネットワークの出力、$t_k$は正解ラベルとなるインデックスだけが1でその他は0であるone-hotベクトル。そのため、実質的に上記の式は、正解ラベルが1に対応する出力の自然対数を計算するだけと言える。

実装上は、$log(0)$がマイナス無限大になってしまうので以下のように微小な値を加えて対策をする点がtip。

In [ ]:
import numpy as np

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))

t7 = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
t2 = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print( cross_entropy_error(np.array(y), np.array(t2)) )
print( cross_entropy_error(np.array(y), np.array(t7)) )

### Gradient

In [ ]:
def function_2(x):
    # df/dx0 = 2*x0
    # df/dx1 = 2*x1
    return x[0]**2 + x[1]**2

def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad

print( numerical_gradient(function_2, np.array([3.0, 4.0])) )
print( numerical_gradient(function_2, np.array([0.0, 2.0])) )

In [ ]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr*grad
    
    return x

print(gradient_descent(function_2, np.array([3.0, 4.0])))

### Gradient Descent in NN

In [ ]:
import sys, os
sys.path.append("./dlscratch")
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss
    
net = simpleNet()
print(net.W)
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
np.argmax(p)
t = np.array([0, 0, 1])
net.loss(x, t)

In [ ]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

#### SGD

ニューラルネットワークの学習は勾配降下法によってパラメータを更新する方法だが、ここで使用するデータはミニバッチとして無作為に選ばれたデータを使用している事から、確率敵勾配降下法（Stochastic Gradient Descent）と呼ばれる。「確率的」の理由はここにある。つまり、「確率的に無作為に選び出した」という意味。

### Two Layers NN

In [ ]:
import sys, os
sys.path.append("./dlscratch")
import numpy as np
from common.functions import softmax, cross_entropy_error, sigmoid
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # initialize weight
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [ ]:
import numpy as np
from dataset.mnist import load_mnist
import matplotlib.pyplot as plt

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

print("start training")
x = np.linspace(0, 1, iters_num)
for i in range(iters_num):
    # get a mini batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # compute gradient
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # update parameters
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # record learning
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    print(loss)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(train_loss_list, color='lightblue', linewidth=3)
    plt.show()

### Backpropagation

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy
    

class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    
# example
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer(apple_price, tax)

print(price)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

TypeError: 'MulLayer' object is not callable

### Activation Functions Layer

* ReLU
* Sigmoid

In [8]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)  # 便利なので覚えておきたい使い方
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    


[[False  True]
 [ True False]]
[[1. 0.]
 [0. 3.]]
